# Methods Appendix

In [1]:
import os
os.getcwd()

PROJECT_ROOT = "/Users/bradykennedy/Documents/GitHub/ClassAndClimate"
os.chdir(PROJECT_ROOT)
print("Working directory set to:", os.getcwd())

Working directory set to: /Users/bradykennedy/Documents/GitHub/ClassAndClimate


In [7]:
import pandas as pd
import statsmodels.formula.api as smf

In [4]:
df = pd.read_csv("data/processed/analysis_model_2024.csv")
model_cols = [
    "clean_energy_support", "weight", "homeowner",
    "pid7", "ideo5", "income_group", "education_group",
    "age_group", "gender4", "race_eth"
]
m = df.dropna(subset=model_cols).copy()
len(df), len(m)

(60000, 56167)

## Weighting

All descriptive estimates and WLS regressions are weighted using common weight provided by CES. Robust (HC1) standard errors are reported.

## Outcome Variable 

Index ∈ [0,1] constructed as the mean of multiple policy-support indicators. 

Using an index reduces noise from any single item and captures a broader policy orientation.

In [6]:
df["clean_energy_support"].describe()
df["n_index_items"].value_counts().head()

n_index_items
3    59992
2        7
1        1
Name: count, dtype: int64

## Predictors

Politics

    party ID (pid7)

    ideology (ideo5)

Class / position

    homeownership (binary)

    income group (low/middle/high)

    education (collapsed to education_simple: hs_or_less / some_college / college_plus)

Controls

    age group

    gender

    race/ethnicity

## Modeling Specs

Weighted least squares: clean_energy_support regressed on party ID, ideology, income group, education, homeownership, and demographics.

In [9]:
formula = """
clean_energy_support
~ homeowner
+ C(pid7, Treatment(reference=1))
+ C(ideo5, Treatment(reference=1))
+ C(income_group, Treatment(reference="low"))
+ C(education_group, Treatment(reference="hs_or_less"))
+ C(age_group, Treatment(reference="18_29"))
+ C(gender4, Treatment(reference=1))
+ C(race_eth, Treatment(reference="white_nonhisp"))
"""

model_wls = smf.wls(formula, data=m, weights=m["weight"]).fit()
m_robust = model_wls.get_robustcov_results(cov_type="HC1")

m_robust.summary().tables[0]

Dep. Variable:,clean_energy_support,R-squared:,0.458
Model:,WLS,Adj. R-squared:,0.457
Method:,Least Squares,F-statistic:,856.7
Date:,"Thu, 22 Jan 2026",Prob (F-statistic):,0.00
Time:,14:41:46,Log-Likelihood:,-33470.
No. Observations:,56167,AIC:,6.699e+04
Df Residuals:,56140,BIC:,6.724e+04
Df Model:,26,,
Covariance Type:,HC1,,


## Outputs & Reproducibility

Outputs are saved accordingly:

exports/figures/*.png

exports/tables/*.html

SQL scripts create data/processed/analysis_model_2024.csv

analysis notebook: 01_analysis.ipynb

results brief: 02_results_brief.ipynb



## Reproducibility

1. run all DuckDB SQL scripts in sql/ folder

2. run notebooks/01_analysis.ipynb

3. run report/brief.ipynb

(Optional) run notebooks/02_results_brief.ipynb

(Optional) run notebooks/03_methods_appendix.ipynb